In [27]:
import pandas as pd

In [28]:
file_root = "../../data/raw/"

In [29]:
# The location of hospitals provided by the Department of Homeland Security.
file_name = "Hospitals.csv"

df = pd.read_csv(file_root + file_name, header=0)

In [30]:
print("Column names:", df.columns.values)
print("Total number of rows:", df.shape[0])

Column names: ['X' 'Y' 'OBJECTID' 'ID' 'NAME' 'ADDRESS' 'CITY' 'STATE' 'ZIP' 'ZIP4'
 'TELEPHONE' 'TYPE' 'STATUS' 'POPULATION' 'COUNTY' 'COUNTYFIPS' 'COUNTRY'
 'LATITUDE' 'LONGITUDE' 'NAICS_CODE' 'NAICS_DESC' 'SOURCE' 'SOURCEDATE'
 'VAL_METHOD' 'VAL_DATE' 'WEBSITE' 'STATE_ID' 'ALT_NAME' 'ST_FIPS' 'OWNER'
 'TTL_STAFF' 'BEDS' 'TRAUMA' 'HELIPAD']
Total number of rows: 8112


In [31]:
# There is no documentation for this dataset.
# Population looks like a very small number. Perhaps it refers to hospital capacity or staff.
df.groupby("COUNTYFIPS").POPULATION.max()

COUNTYFIPS
1001     107
1003     176
1005      74
1007      35
1009      40
        ... 
72145    164
72147   -999
72153    105
78010    188
78030    169
Name: POPULATION, Length: 2588, dtype: int64

In [32]:
# Types of hospital
df.groupby("TYPE").nunique()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
TYPE,,,,,,,,,,,,,,,,,,,,,
CHILDREN,164,164,164,164,157,161,121,40,151,29,...,53,73,127,45,40,5,1,111,12,3
CHRONIC DISEASE,12,12,12,12,12,12,11,7,11,6,...,7,10,10,4,7,4,1,11,1,2
CRITICAL ACCESS,1280,1279,1281,1269,1245,1269,1118,42,1248,222,...,112,702,1086,387,42,7,1,72,9,2
GENERAL ACUTE CARE,4281,4279,4291,4239,4112,4211,2278,57,3622,578,...,222,2207,3458,1204,56,7,1,723,24,3
LONG TERM CARE,387,387,387,383,380,385,295,42,362,46,...,88,276,303,119,42,6,1,120,5,2
MILITARY,285,284,285,282,279,281,247,53,267,166,...,49,268,221,101,53,2,1,114,5,2
PSYCHIATRIC,995,995,997,983,971,979,653,52,890,181,...,119,484,748,264,52,6,1,241,5,3
REHABILITATION,460,460,460,455,449,456,350,47,427,42,...,128,240,374,119,47,6,1,120,5,3
SPECIAL,215,215,216,213,213,213,151,34,188,21,...,71,100,163,41,34,6,1,83,6,3


In [33]:
# Rename and select relevant columns
columns_map = {
    "COUNTYFIPS": "fips",
    "LATITUDE": "latitude",
    "LONGITUDE": "longitude",
    "TYPE": "type",
    "BEDS": "number_of_beds",
}

df = df.rename(columns=columns_map)
df = df[columns_map.values()]
df

,fips,latitude,longitude,type,number_of_beds
0,1083,34.804164,-86.978407,GENERAL ACUTE CARE,71
1,1053,31.017024,-87.478799,GENERAL ACUTE CARE,51
2,1101,32.376399,-86.174930,GENERAL ACUTE CARE,150
3,1101,32.328154,-86.277549,GENERAL ACUTE CARE,432
4,1097,30.624808,-88.178008,PSYCHIATRIC,60
...,...,...,...,...,...
8107,34023,40.500067,-74.347278,GENERAL ACUTE CARE,-999
8108,8041,38.968781,-104.754827,CHILDREN,-999
8109,22047,30.266626,-91.219417,GENERAL ACUTE CARE,-999
8110,39023,39.939755,-83.811348,GENERAL ACUTE CARE,-999


In [34]:
# Write out the cleaned data
df.to_csv("../../data/processed/hospitals_location.csv", index=False)

In [35]:
# Read in the SVI dataset to get population counts
svi = pd.read_csv("../../data/processed/social_vulnerability_index.csv")
svi = svi[["fips", "population"]].set_index("fips")
svi

,population
fips,
1001,55639
1003,218289
1005,25026
1007,22374
1009,57755
...,...
56037,43352
56039,23356
56041,20374


In [36]:
total_hospitals = df.groupby("fips").size().to_frame("total_hospitals").reset_index()
total_hospitals

,fips,total_hospitals
0,1001,1
1,1003,4
2,1005,1
3,1007,1
4,1009,1
...,...,...
2583,72145,2
2584,72147,1
2585,72153,1
2586,78010,1


In [37]:
total_hospitals = total_hospitals.set_index("fips").join(svi).reset_index()
total_hospitals["hospitals_per_100000"] = total_hospitals["total_hospitals"] / (
    total_hospitals["population"] / 100000
)
total_hospitals = total_hospitals.drop(columns=["population"])
total_hospitals

,fips,total_hospitals,hospitals_per_100000
0,1001,1,1.797300
1,1003,4,1.832433
2,1005,1,3.995844
3,1007,1,4.469473
4,1009,1,1.731452
...,...,...,...
2583,72145,2,NaN
2584,72147,1,NaN
2585,72153,1,NaN
2586,78010,1,NaN


In [38]:
total_hospitals.to_csv("../../data/processed/hospitals_total.csv", index=False)

In [39]:
# The original dataset marks N/A values for number of beds as -999. Let's filter them out.
total_beds = df[df["number_of_beds"] != -999]
total_beds = (
    total_beds.groupby("fips")["number_of_beds"]
    .sum()
    .to_frame("total_beds")
    .reset_index()
)
total_beds

,fips,total_beds
0,1001,107
1,1003,432
2,1005,74
3,1007,35
4,1009,40
...,...,...
2570,72127,2583
2571,72145,314
2572,72153,105
2573,78010,188


In [40]:
total_beds = total_beds.set_index("fips").join(svi).reset_index()
total_beds["hospital_beds_per_100000"] = total_beds["total_beds"] / (
    total_beds["population"] / 100000
)
total_beds = total_beds.drop(columns=["population"])
total_beds

,fips,total_beds,hospital_beds_per_100000
0,1001,107,192.311149
1,1003,432,197.902780
2,1005,74,295.692480
3,1007,35,156.431572
4,1009,40,69.258073
...,...,...,...
2570,72127,2583,NaN
2571,72145,314,NaN
2572,72153,105,NaN
2573,78010,188,NaN


In [41]:
total_beds.to_csv("../../data/processed/hospital_beds_total.csv", index=False)